In [1]:
# -*- coding: utf-8 -*-
# This code is written in Python 3.7. To install the required packages, run the following commands:
# pip install pandas numpy matplotlib seaborn scikit-learn sympy
# This code is applicable to the Simargl 2022 dataset.
# implemented Advanced Ensamble model (LightGBM) for classification.

import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import LabelEncoder


/tmp/ipykernel_1728964/3663230059.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-02-26 15:44:08.898906: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-26 15:44:08.934941: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 15:

In [ ]:
# Load the cleaned Simargl 2022 dataset
df = pd.read_csv(
    '/home/ibibers@ads.iu.edu/IDS_Datasets/Combined_datasets/Simargl_cleaned_dataset.csv')
# tf.keras.backend.clear_session()

In [2]:
dropped_df = ['ALERT']
X = df.drop(dropped_df, axis=1)
y = df['ALERT']

# Label encoding for the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print(df['ALERT'].value_counts())

In [5]:
# Initial train test split set and split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# # Convert data to TensorFlow tensors
# X_train_tf = tf.convert_to_tensor(X_train.values, dtype=tf.float32)
# y_train_tf = tf.convert_to_tensor(pd.factorize(y_train)[0], dtype=tf.int64)
# X_test_tf = tf.convert_to_tensor(X_test.values, dtype=tf.float32)

# Initialize MirroredStrategy for GPU acceleration
strategy = tf.distribute.MirroredStrategy()



# Train the models
with strategy.scope():
  
    # Light GBM Light GBM beats all the other algorithms when the dataset is extremely large.
    train_data = lgb.Dataset(X_train, label=y_train)
    # define parameters
    # params = {'learning_rate':0.001}
    params = {
        'learning_rate': 0.001,
        'num_leaves': 31,
        'max_depth': -1,
        'min_child_samples': 20,
    }

    # lgb_model= lgb.train(params, train_data, 100)
    lgb_model = lgb.train(params, train_data, num_boost_round=200)

    # Make predictions on the test set
    y_pred = lgb_model.predict(X_test)

    # Threshold the predicted probabilities at 0.5 to obtain binary predictions
    y_pred_encoded = [1 if prob >= 0.5 else 0 for prob in y_pred]




In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred_encoded)
print(f"Accuracy: {accuracy}")

In [ ]:
# Use inverse_transform to get original labels
y_pred = label_encoder.inverse_transform(y_pred_encoded)
y_test_labels = label_encoder.inverse_transform(y_test)

In [ ]:
# Classification Report
print("\nClassification Report:")
print(classification_report(y_test_labels, y_pred))

NameError: name 'y_pred_encoded' is not defined

In [ ]:

# Confusion matrix with string labels
conf_mat = confusion_matrix(y_test_labels, y_pred)

# Get unique class labels from y_test and y_pred_encoded
unique_labels = np.unique(np.concatenate(
    (y_test_labels, y_pred)))

# Plotting the Confusion Matrix with class labels
plt.figure(figsize=(6, 6))
sns.heatmap(conf_mat, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=unique_labels, yticklabels=unique_labels)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()